In [189]:
import pandas as pd
import numpy as np

In [190]:
pd.read_excel("BTC-USD.xlsx").head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800.0
1,2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200.0
2,2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700.0
3,2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600.0
4,2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100.0


In [191]:
BTC = pd.read_excel("BTC-USD.xlsx")

In [192]:
BTC_modify = pd.DataFrame(BTC).dropna()

In [193]:
import numpy
import talib

close = numpy.random.random(30)
print(type(close))
print(talib.SMA(close, timeperiod=5))

close = BTC_modify["Close"].to_numpy()
print(type(close))
print(talib.SMA(close, timeperiod=5))

<class 'numpy.ndarray'>
[       nan        nan        nan        nan 0.56719895 0.66041864
 0.59142341 0.414493   0.30856202 0.37895233 0.2487966  0.28913917
 0.29264332 0.27967065 0.39603181 0.50941212 0.52696079 0.57065553
 0.55060428 0.38906988 0.41428756 0.40323955 0.38796263 0.48403069
 0.460749   0.29340419 0.35771476 0.36320981 0.25263029 0.39220966]
<class 'numpy.ndarray'>
[          nan           nan           nan ... 43333.1203126 42137.6304688
 40795.0625   ]


In [194]:
# note that all ndarrays must be the same length!
inputs = {
    'open': BTC_modify["Open"].to_numpy(),
    'high': BTC_modify["High"].to_numpy(),
    'low': BTC_modify["Low"].to_numpy(),
    'close': BTC_modify["Close"].to_numpy(),
    'volume': BTC_modify["Volume"].to_numpy()
}

from talib.abstract import *

SMA(inputs, timeperiod=5) # calculate on close prices by default
# print(output)
# print(len(list(output)))
# print(pd.DataFrame(output).dropna())
# print(pd.DataFrame(output).dropna().shape)

array([          nan,           nan,           nan, ..., 43333.1203126,
       42137.6304688, 40795.0625   ])

In [195]:
BTC_modify["5ma"]=SMA(inputs,timeperiod=5)

In [196]:
BTC_modify["7ma"]=None
BTC_modify["15ma"]=None
BTC_modify["21ma"]=None
BTC_modify["7ma"]=SMA(inputs,timeperiod=7)
BTC_modify["15ma"]=SMA(inputs,timeperiod=15)
BTC_modify["21ma"]=SMA(inputs,timeperiod=21)

In [197]:
BTC_input=pd.DataFrame(BTC_modify).dropna()

In [198]:
BTC_input["buy_day"]=0
BTC_input["cum_day"]=0
BTC_input["sell_day"]=0
BTC_input["number"]=0
BTC_input["value"]=0
BTC_input["period_return"]=0
BTC_input["holding_period"]=0
BTC_input["annual_return"]=0
BTC_input["diff"]=0
BTC_input["limit"]=-0.05
BTC_input["sell1"]=0

In [199]:
#Strategy of trading
#買: 7MA>15MA or 7M>21MA
#賣: 21MA<15MA and 跌幅(by today open & adj close)5%隔天放空

In [200]:
#sell1
for i in range(1, len(BTC_input)):
    upper=BTC_input["Adj Close"].iloc[i]-BTC_input["Open"].iloc[i]
    BTC_input["diff"].iloc[i]=upper/BTC_input["Open"].iloc[i]

for i in range(1, len(BTC_input)):
    if(BTC_input["diff"].iloc[i]<BTC_input["limit"].iloc[i]):
        BTC_input["sell1"].iloc[i]=1

C:\Users\user\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [201]:
for i in range(1, BTC_input.shape[0]):
    if((BTC_input["7ma"].iloc[i] > BTC_input["15ma"].iloc[i])\
    or(BTC_input["7ma"].iloc[i] > BTC_input["21ma"].iloc[i]))\
    and(BTC_input["cum_day"].iloc[i-1]==0):
        BTC_input["buy_day"].iloc[i]  = 1
        BTC_input["cum_day"].iloc[i]  = 1
        BTC_input["sell_day"].iloc[i] = 0
    elif(BTC_input["21ma"].iloc[i] < BTC_input["15ma"].iloc[i])and(BTC_input["cum_day"].iloc[i-1]==1)and(BTC_input["sell1"].iloc[i+1]==1):
        BTC_input["buy_day"].iloc[i]  = 0
        BTC_input["cum_day"].iloc[i]  = 0
        BTC_input["sell_day"].iloc[i] = 1
    elif(BTC_input["cum_day"].iloc[i-1]>0):
        BTC_input["buy_day"].iloc[i]  = 0
        BTC_input["cum_day"].iloc[i]  = 1
        BTC_input["sell_day"].iloc[i] = 0

In [202]:
BTC_input.to_excel("progb.xlsx")

In [203]:
#add
initial=10000
for i in range(1, BTC_input.shape[0]):
#buy signal
    if(BTC_input["buy_day"].iloc[i]==1)and(BTC_input["cum_day"].iloc[i]==1)and(BTC_input["sell_day"].iloc[i]==0):
        BTC_input["number"].iloc[i] =initial/BTC_input["Adj Close"].iloc[i]
        BTC_input["value"].iloc[i]=BTC_input["number"].iloc[i]*BTC_input["Adj Close"].iloc[i]
        BTC_input["period_return"].iloc[i]=round((BTC_input["number"].iloc[i]*BTC_input["Adj Close"].iloc[i]-initial)/initial,4)
#sell signal
    elif(BTC_input["buy_day"].iloc[i]==0)and(BTC_input["cum_day"].iloc[i]==0)and(BTC_input["sell_day"].iloc[i]==1):
        BTC_input["number"].iloc[i]=0
        BTC_input["value"].iloc[i]=BTC_input["number"].iloc[i-1]*BTC_input["Adj Close"].iloc[i]
        BTC_input["period_return"].iloc[i]=round((BTC_input["number"].iloc[i-1]*BTC_input["Adj Close"].iloc[i]-initial)/initial,4)
#hoding
    elif(BTC_input["buy_day"].iloc[i]==0)and(BTC_input["cum_day"].iloc[i]==1)and(BTC_input["sell_day"].iloc[i]==0):
        BTC_input["number"].iloc[i]=BTC_input["number"].iloc[i-1]
        BTC_input["value"].iloc[i]=BTC_input["number"].iloc[i]*BTC_input["Adj Close"].iloc[i]
        BTC_input["period_return"].iloc[i]=round((BTC_input["number"].iloc[i]*BTC_input["Adj Close"].iloc[i]-initial)/initial,4)

In [204]:
BTC_input.to_excel("progb0604.xlsx")

In [205]:
start = 0
end = 0
for i in range(1, len(BTC_input)):
    if(BTC_input["buy_day"].iloc[i]==1):
        BTC_input["holding_period"].iloc[i] = 1
    elif((BTC_input["cum_day"].iloc[i] > 0) or (BTC_input["sell_day"].iloc[i] == 1))and(BTC_input["buy_day"].iloc[i] == 0):
        BTC_input["holding_period"].iloc[i] = BTC_input["holding_period"].iloc[i-1] +1
BTC_input.head(6)

,Date,Open,High,Low,Close,Adj Close,Volume,5ma,7ma,15ma,...,cum_day,sell_day,number,value,period_return,holding_period,annual_return,diff,limit,sell1
20,2014-10-07,330.584015,339.247009,320.481995,336.187012,336.187012,49199900.0,335.030804,347.691572,376.529864,...,0,0,0.0,0.0,0.0,0,0,0.000000,-0.05,0
21,2014-10-08,336.115997,354.364014,327.187988,352.940002,352.940002,54736300.0,333.716406,343.309431,371.006466,...,0,0,0.0,0.0,0.0,0,0,0.050054,-0.05,0
22,2014-10-09,352.747986,382.726013,347.687012,365.026001,365.026001,83641104.0,340.948407,341.874289,367.127867,...,0,0,0.0,0.0,0.0,0,0,0.034807,-0.05,0
23,2014-10-10,364.687012,375.066986,352.963013,361.562012,361.562012,43665700.0,349.158807,342.167149,363.793734,...,0,0,0.0,0.0,0.0,0,0,-0.008569,-0.05,0
24,2014-10-11,361.362000,367.191010,355.950989,362.299011,362.299011,13345200.0,355.602808,346.943294,360.985335,...,0,0,0.0,0.0,0.0,0,0,0.002593,-0.05,0
25,2014-10-12,362.605988,379.433014,356.144012,378.549011,378.549011,17552800.0,364.075207,355.234580,359.587270,...,0,0,0.0,0.0,0.0,0,0,0.043968,-0.05,0


In [206]:
for i in range(1, len(BTC_input)):
    if(BTC_input["buy_day"].iloc[i]==1)or(BTC_input["sell_day"].iloc[i]==1)or(BTC_input["cum_day"].iloc[i]==1):
        BTC_input["annual_return"].iloc[i]= (1+BTC_input["period_return"].iloc[i])**(365/BTC_input["holding_period"].iloc[i])-1

C:\Users\user\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [207]:
BTC_input.head(25)

,Date,Open,High,Low,Close,Adj Close,Volume,5ma,7ma,15ma,...,cum_day,sell_day,number,value,period_return,holding_period,annual_return,diff,limit,sell1
20,2014-10-07,330.584015,339.247009,320.481995,336.187012,336.187012,49199900.0,335.030804,347.691572,376.529864,...,0,0,0.000000,0.000000,0.0000,0,0.000000,0.000000,-0.05,0
21,2014-10-08,336.115997,354.364014,327.187988,352.940002,352.940002,54736300.0,333.716406,343.309431,371.006466,...,0,0,0.000000,0.000000,0.0000,0,0.000000,0.050054,-0.05,0
22,2014-10-09,352.747986,382.726013,347.687012,365.026001,365.026001,83641104.0,340.948407,341.874289,367.127867,...,0,0,0.000000,0.000000,0.0000,0,0.000000,0.034807,-0.05,0
23,2014-10-10,364.687012,375.066986,352.963013,361.562012,361.562012,43665700.0,349.158807,342.167149,363.793734,...,0,0,0.000000,0.000000,0.0000,0,0.000000,-0.008569,-0.05,0
24,2014-10-11,361.362000,367.191010,355.950989,362.299011,362.299011,13345200.0,355.602808,346.943294,360.985335,...,0,0,0.000000,0.000000,0.0000,0,0.000000,0.002593,-0.05,0
25,2014-10-12,362.605988,379.433014,356.144012,378.549011,378.549011,17552800.0,364.075207,355.234580,359.587270,...,0,0,0.000000,0.000000,0.0000,0,0.000000,0.043968,-0.05,0
26,2014-10-13,377.920990,397.226013,368.897003,390.414001,390.414001,35221400.0,371.570007,363.853864,360.469470,...,1,0,25.613836,10000.000000,0.0000,1,0.000000,0.033057,-0.05,0
27,2014-10-14,391.691986,411.697998,391.324005,400.869995,400.869995,38491500.0,378.738806,373.094290,362.163002,...,1,0,25.613836,10267.818110,0.0268,2,123.787030,0.023432,-0.05,0
28,2014-10-15,400.954987,402.226990,388.765991,394.773010,394.773010,25267100.0,385.381006,379.070434,362.684936,...,1,0,25.613836,10111.650939,0.0112,3,2.877119,-0.015418,-0.05,0
29,2014-10-16,394.518005,398.807007,373.070007,382.556000,382.556000,26990000.0,389.432403,381.574720,362.614337,...,1,0,25.613836,9798.726455,-0.0201,4,-0.843204,-0.030321,-0.05,0


In [208]:
annual_return=1
annual_period_return=1
trading=0

for i in range(0,BTC_input.shape[0]):
    if(BTC_input["sell_day"].iloc[i]==1):
        annual_return*=(1+BTC_input["annual_return"].iloc[i])
        annual_period_return*=(1+BTC_input["period_return"].iloc[i])
        trading+=1

final_return=annual_return**(1/trading)-1
final_period_return=annual_period_return**(1/trading)-1
print("交易次數：",trading,"年化報酬",final_return,"期間年化報酬",final_period_return)

交易次數： 79 年化報酬 6.731365471531088 期間年化報酬 0.14707282907062358


In [209]:
annual_return = 1
annual_period_return = 1
trading = 0

for i in range(0, BTC_input.shape[0]):
    if(BTC_input["sell_day"].iloc[i]==1):
        annual_return *= (1 + BTC_input["annual_return"].iloc[i] )
        annual_period_return *= (1 + BTC_input["period_return"].iloc[i] )
        trading+=1

final_return=annual_return**(1/trading)-1
final_period_return=annual_period_return**(1/trading)-1
print("交易次數:", trading, "年化報酬:", final_return, "期間年化報酬:", final_period_return)

交易次數: 79 年化報酬: 6.731365471531088 期間年化報酬: 0.14707282907062358


In [210]:
BTC_input.to_excel("AT_Final.xlsx")